# FloPy

### MT3D-USGS Example

Demonstrates functionality of the flopy MT3D-USGS module using the 'Crank-Nicolson' example distributed with MT3D-USGS.  

#### Problem description:

* Grid dimensions: 1 Layer, 3 Rows, 650 Columns
* Stress periods: 12
* Units are in seconds and days
* Flow package: UPW
* Stress packages: SFR, GHB, UZF
* Solvers: NWT, GCG

In [20]:
import sys
import os
import flopy.utils.binaryfile as bf
import flopy
import numpy as np
import math
from io import StringIO

modelpth = os.path.join('data')
modelname = 'CrnkNic'
mfexe = 'mfnwt'

# Make sure modelpth directory exists
if not os.path.exists(modelpth):
    os.mkdir(modelpth)

# Instantiate MODFLOW object in flopy
mf = flopy.modflow.Modflow(modelname=modelname, exe_name=mfexe, model_ws=modelpth, version='mfnwt')


Set up model discretization

In [21]:
Lx = 650.0 
Ly = 15
nrow = 3
ncol = 650
nlay = 1

delr = Lx / ncol
delc = Ly / nrow

xmax = ncol * delr
ymax = nrow * delc

X, Y = np.meshgrid(np.linspace(delr / 2, xmax - delr / 2, ncol),    
                   np.linspace(ymax - delc / 2, 0 + delc / 2, nrow))


Instantiate output control (oc) package for MODFLOW-NWT

In [22]:
# Output Control: Create a flopy output control object
oc = flopy.modflow.ModflowOc(mf)                      

Instantiate solver package for MODFLOW-NWT

In [23]:
# Newton-Rhapson Solver: Create a flopy nwt package object

headtol = 1.0E-4 
fluxtol = 5      
maxiterout = 5000
thickfact = 1E-06
linmeth = 2      
iprnwt = 1       
ibotav = 1       

nwt = flopy.modflow.ModflowNwt(mf, headtol=headtol, fluxtol=fluxtol, maxiterout=maxiterout,       
                               thickfact=thickfact, linmeth=linmeth, iprnwt=iprnwt, ibotav=ibotav,
                               options='SIMPLE')                                          


Instantiate discretization (DIS) package for MODFLOW-NWT

In [24]:
# The equations for calculating the ground elevation in the 1 Layer CrnkNic model.  
# Although Y isn't used, keeping it here for symetry

def topElev(X, Y):
    return 100. - (np.ceil(X)-1) * 0.03

grndElev = topElev(X, Y)
bedRockElev = grndElev - 3.

Steady = [False, False, False]                                                                  
nstp = [1, 1, 1]                                                                        
tsmult = [1., 1., 1.]

# Stress periods extend from (12AM-8:29:59AM); (8:30AM-11:30:59AM); (11:31AM-23:59:59PM)
perlen = [30600, 10800, 45000]                                                                    
                                                                                       
# Create the discretization object
# itmuni = 1 (seconds); lenuni = 2 (meters)
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, nper=3, delr=delr, delc=delc,     
                               top=grndElev, botm=bedRockElev, laycbd=0, itmuni=1, lenuni=2,
                               steady=Steady, nstp=nstp, tsmult=tsmult, perlen=perlen) 


Instantiate upstream weighting (UPW) flow package for MODFLOW-NWT

In [25]:
# UPW parameters 
# UPW must be instantiated after DIS.  Otherwise, during the mf.write_input() procedures,   
# flopy will crash.


laytyp = 1   
layavg = 2   
chani = 1.0  
layvka = 1
iphdry = 0
hk = 0.1
hani = 1
vka = 1.
ss = 0.000001
sy = 0.20
hdry = -888

upw = flopy.modflow.ModflowUpw(mf, laytyp=laytyp, layavg=layavg, chani=chani, layvka=layvka,
                               ipakcb=53, hdry=hdry, iphdry=iphdry, hk=hk, hani=hani,
                               vka=vka, ss=ss, sy=sy)
                                 


Instantiate basic (BAS or BA6) package for MODFLOW-NWT

In [26]:
# Create a flopy basic package object                                               
def calc_strtElev(X, Y):             
    return 99.5 - (np.ceil(X)-1) * 0.0001

ibound = np.ones((nlay, nrow, ncol))
ibound[:,0,:] *= -1
ibound[:,2,:] *= -1

strtElev = calc_strtElev(X, Y)

bas = flopy.modflow.ModflowBas(mf, ibound=ibound, hnoflo=hdry, strt=strtElev)


Instantiate streamflow routing (SFR2) package for MODFLOW-NWT

In [27]:
# Streamflow Routing Package: Try and set up with minimal options in use
# 9 11 IFACE # Data Set 1: ISTCB1  ISTCB2

nstrm = ncol  
nss = 6       
const = 1.0   
dleak = 0.0001
istcb1 = -10  
istcb2 = 11   
isfropt = 1   

segment_data = None
channel_geometry_data = None
channel_flow_data = None
dataset_5 = None
reachinput = True


# The next couple of lines set up the reach_data for the 30x100 hypothetical model.
# Will need to adjust the row based on which grid discretization we're doing.
# Ensure that the stream goes down one of the middle rows of the model.

strmBed_Elev = 98.75 - (np.ceil(X[1,:])-1) * 0.0001

s1 = u"k,i,j,iseg,ireach,rchlen,strtop,slope,strthick,strhc1\n"
iseg = 0
irch = 0

for y in xrange(ncol):
    if y <= 37:
        if iseg == 0:
            irch = 1
        else:
            irch += 1
        iseg = 1
        strhc1 = 1.0e-10
    elif y <= 104:
        if iseg == 1:
            irch = 1
        else:
            irch += 1
        iseg = 2
        strhc1 = 1.0e-10
    elif y <= 280:
        if iseg == 2:
            irch = 1
        else:
            irch += 1
        iseg = 3
        strhc1 = 2.946219199e-6
    elif y <= 432:
        if iseg == 3:
            irch = 1
        else:
            irch += 1
        iseg = 4
        strhc1 = 1.375079882e-6
    elif y <= 618:
        if iseg == 4:
            irch = 1
        else:
            irch += 1
        iseg = 5
        strhc1 = 1.764700062e-6
    else:
        if iseg == 5:
            irch = 1
        else:
            irch += 1
        iseg = 6
        strhc1 = 1e-10
    
    # remember that lay, row, col need to be zero-based and are adjusted accordingly by flopy
    #    layer +  row      +      col     +       iseg      +      irch      +     rchlen      +            strtop          +       slope       +     strthick    +     strmbed K
    s1 += '0,' + str(1) + ',' + str(y) + ',' + str(iseg) + ',' + str(irch) + ',' + str(delr) + ',' + str(strmBed_Elev[y]) + ',' + str(0.0001) + ',' + str(0.50) + ',' + str(strhc1)  + u'\n'

s1 = StringIO(s1)
reach_data = np.genfromtxt(s1, delimiter=',',names=True, dtype=[('k', '<f8'), ('i', '<f8'), ('j', '<f8'), ('iseg', '<f8'), ('ireach', '<f8'), ('rchlen', '<f8'), ('strtop', '<f8'), ('slope', '<f8'), ('strthick', '<f8'), ('strhc1', '<f8')])

s2 = StringIO(u"nseg,icalc,outseg,iupseg,nstrpts,   flow,runoff,etsw,pptsw,        roughch,        roughbk,cdpth,fdpth,awdth,bwdth,width1,width2\n \
                   1,    1,     2,     0,      0, 0.0125,   0.0, 0.0,  0.0, 0.082078856000, 0.082078856000,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5\n \
                   2,    1,     3,     0,      0,    0.0,   0.0, 0.0,  0.0, 0.143806300000, 0.143806300000,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5\n \
                   3,    1,     4,     0,      0,    0.0,   0.0, 0.0,  0.0, 0.104569661821, 0.104569661821,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5\n \
                   4,    1,     5,     0,      0,    0.0,   0.0, 0.0,  0.0, 0.126990045841, 0.126990045841,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5\n \
                   5,    1,     6,     0,      0,    0.0,   0.0, 0.0,  0.0, 0.183322283828, 0.183322283828,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5\n \
                   6,    1,     0,     0,      0,    0.0,   0.0, 0.0,  0.0, 0.183322283828, 0.183322283828,  0.0,  0.0,  0.0,  0.0,   1.5,   1.5")
segment_data = np.genfromtxt(s2, delimiter=',',names=True)

# Be sure to convert segment_data to a dictionary keyed on stress period.
segment_data = np.atleast_1d(segment_data)                                                                                                                                                   
segment_data = {0: segment_data,                                                                                                                                                             
                1: segment_data,
                2: segment_data}
                                                                                                                                                                                             
# There are 3 stress periods
dataset_5 = {0: [nss, 0, 0],
             1: [nss, 0, 0],
             2: [nss, 0, 0]}
 
sfr = flopy.modflow.ModflowSfr2(mf, nstrm=nstrm, nss=nss, const=const, dleak=dleak, isfropt=isfropt, istcb2=0,
                                reachinput=True, reach_data=reach_data, dataset_5=dataset_5,
                                segment_data=segment_data, channel_geometry_data=channel_geometry_data)


Instantiate gage package for use with MODFLOW-NWT package

In [28]:
gages = [[1,38,61,1],[2,67,62,1], [3,176,63,1], [4,152,64,1], [5,186,65,1], [6,31,66,1]]
files = ['CrnkNic.gage','CrnkNic.gag1','CrnkNic.gag2','CrnkNic.gag3','CrnkNic.gag4','CrnkNic.gag5',
         'CrnkNic.gag6']
gage = flopy.modflow.ModflowGage(mf, numgage=6, gage_data=gages, filenames = files)

Instantiate streamflow routing (SFR2) package for MODFLOW-NWT

In [29]:
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='CrnkNic.ftl', output_file_header='extended',
                               output_file_format='formatted', package_flows = ['sfr'])

Write the MODFLOW input files

In [31]:
pth = os.getcwd()
print(pth)

mf.write_input()

# run the model
mf.run_model()

D:\EDM_LT\GitHub\flopy_fork\examples\Notebooks
FloPy is using the following executable to run the model: C:\Program Files\USGS\Phast for Windows x64 3.3.7-11094\bin\mfnwt.exe

                                  MODFLOW-NWT-SWR1 
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                             WITH NEWTON FORMULATION
                             Version 1.1.2, 9/15/2016                        
                    BASED ON MODFLOW-2005 Version 1.11.0 08/08/2013                       

                    SWR1 Version 1.04.0 09/15/2016                       

 Using NAME file: CrnkNic.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2017/02/16 13:03:40

 Solving:  Stress period:     1    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Groundwater-Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2017/02/16 13:03:40
 

(True, [])

Now draft up MT3D-USGS input files.  

In [32]:
# Instantiate MT3D-USGS object in flopy
mt = flopy.mt3d.Mt3dms(modflowmodel=mf, modelname=modelname, model_ws=modelpth,
                       version='mt3d-usgs', namefile_ext='mtnam', exe_name='mt3d-usgs.exe',
                       ftlfilename='CrnkNic.ftl', ftlfree=True)

# ask Chris about whether .Mt3dms should be altered to .Mt3dUsgs for promote USGS product?

Instantiate basic transport (BTN) package for MT3D-USGS

In [33]:
btn = flopy.mt3d.Mt3dBtn(mt, sconc=3.7, ncomp=1, prsity=0.2, cinact=-1.0, 
                         thkmin=0.001, nprs=-1, nprobs=10, chkmas=True, 
                         nprmas=10, dt0=180, mxstrn=2500)

D:\EDM_LT\GitHub\flopy_fork\flopy\mt3d\mtbtn.py:462: UserWarning: BTN warning. Laycon has not been set.  A modflow model with a  BCF or LPF package does not exist and laycon was not passed  to the BTN constructor.  Setting laycon to 1 (convertible).
  warnings.warn(s)


Instantiate advection (ADV) package for MT3D-USGS

In [34]:
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=0, percel=1.00, mxpart=5000, nadvfd=1)

Instatiate generalized conjugate gradient solver (GCG) package for MT3D-USGS

In [35]:
gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=10, iter1=50, isolve=3, ncrs=0,
                   accl=1, cclose=1e-6, iprgcg=1)

Instantiate source-sink mixing (SSM) package for MT3D-USGS

In [36]:
# For SSM, need to set the constant head boundary conditions to the ambient concentration
# for all 1,300 constant head boundary cells.

itype = flopy.mt3d.Mt3dSsm.itype_dict()
ssm_data = {}
ssm_data[0] = [(0, 0, 0, 3.7, itype['CHD'])]
ssm_data[0].append((0, 2, 0, 3.7, itype['CHD']))
for i in [0,2]:
    for j in xrange(1, ncol):
        ssm_data[0].append((0, i, j, 3.7, itype['CHD']))

ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=ssm_data)


Instantiate streamflow transport (SFT) package for MT3D-USGS

In [37]:
dispsf = []
for y in xrange(ncol):
    if y <= 37:
        dispsf.append(0.12)
    elif y <= 104:
        dispsf.append(0.15)
    elif y <= 280:
        dispsf.append(0.24)
    elif y <= 432:
        dispsf.append(0.31)
    elif y <= 618:
        dispsf.append(0.40)
    else:
        dispsf.append(0.40)

# Enter a list of the observation points
# Each observation is taken as the last reach within the first 5 segments

seg_len = np.unique(reach_data['iseg'], return_counts=True)
obs_sf = np.cumsum(seg_len[1])
obs_sf = obs_sf.tolist()

# The last reach is not an observation point, therefore drop
obs_sf.pop(-1)

# In the first and last stress periods, concentration at the headwater is 3.7
sf_stress_period_data = {0: [1, 0, 3.7],
                         1: [1, 0, 11.4],
                         2: [1, 0, 3.7]}

gage_output = [None, None, 'CrnkNic.sftobs']

sft = flopy.mt3d.Mt3dSft(mt, nsfinit=650, mxsfbc=650, icbcsf=81, ioutobs=82,
                         isfsolv=1, cclosesf=1.0E-6, mxitersf=10, crntsf=1.0, iprtxmd=0,
                         coldsf=3.7, dispsf=dispsf, nobssf=5, obs_sf=obs_sf,
                         sf_stress_period_data = sf_stress_period_data,
                         filenames=gage_output)

sft.dispsf.format.fortran = "(10E15.6)"

Write the MT3D-USGS input files

In [38]:
mt.write_input()

# run the model
mt.run_model()

FloPy is using the following executable to run the model: C:\Program Files\USGS\Phast for Windows x64 3.3.7-11094\bin\mt3d-usgs.exe

 MT3D-USGS - Modular 3D Multi-Species Transport Model [Ver 1.00.00] 
 and based on MT3DMS. MT3D-USGS developed in cooperation by 
 S.S. Papadopulos & Associates and the U.S. Geological Survey

 Using NAME File: CrnkNic.mtnam

 STRESS PERIOD NO.    1

 TIME STEP NO.    1
 FROM TIME =   0.0000     TO    30600.    

 Transport Step:    1   Step Size:   180.0     Total Elapsed Time:   180.00    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Outer Iter.  2  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    2   Step Size:   180.0     Total Elapsed Time:   360.00    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Outer Iter.  2  Inner Iter.  1:  Max. DC =   0.000      [K,I,J]    1    1    1
 Transport Step:    3   Step Size:   180.0     Total Elapsed Time:   540

(False, [])

complete!